# Lab 5: Naive Bayes

In this lab, we will implement a Naive Bayes classifier for the task of spam filtering. We will first employ a classifier included in the `scikit-learn` package and then, we will implement our own classifier. 

Naive Bayes is one of the simplest Bayesian inspired methods. It is a supervised learning method and it is very popular in text classification tasks. One prominent application is as a spam filter for emails.

In this lab, we will employ a subset of the `Enron Email` dataset. The dataset contains emails generated by employees of the Enron Corporation. Specifically, the subset contains the emails of a single Enron employee (farmer-d) who had a fairly large mailbox. The dataset can be downloaded from the following link (http://www.aueb.gr/users/ion/data/enron-spam/preprocessed/enron1.tar.gz).

You are given a function that reads the dataset from the disk and returns two lists. The first contains the emails themselves, while the second contains their class labels ($1$ for ham and $-1$ for spam).

In [1]:
from os import listdir

def read_enron():
    spam_files = listdir("enron1/spam")
    ham_files = listdir("enron1/ham")

    emails = []
    y = []
    for spam_file in spam_files:
        f = open("enron1/spam/"+spam_file, 'r')
        emails.append(f.read())
        y.append(-1)
        f.close()

    for ham_file in ham_files:
        f = open("enron1/ham/"+ham_file, 'r')
        emails.append(f.read())
        y.append(1)
        f.close()
        
    return emails, y

Run the function to load the data from the disk. Print the first email. Compute how many spam and how many ham emails are contained in the dataset.

In [3]:
#your code here


To evaluate the performance of the Naive Bayes classifier, we will use $k$-fold cross-validation with $k$ = 10.
According to this method, the original dataset is randomly split into $k$ subsets of equal size. One of
the $k$ subsets plays the role of the test set, while the remaining $k − 1$ subsets act as the training set.
The process is repeated $k$ times, and each of the $k$ subsets is used exactly once as the test set and the
remaining $k − 1$ as part of the training set. The $k$ results can then be combined together to produce an
overall evaluation of the model. To perform cross-validation, we will employ the `StratifiedKFold` object provided by `scikit-learn` (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) which returns stratified folds, i.e. the folds are made by preserving the percentage of samples for each class.

At each iteration of the $k$-fold cross-validation the original dataset is split into a training and a test set. To classify the data of the test set, we will use the multinomial Naive Bayes classifier of `scikit-learn` (http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html). To train the classifier (i.e. compute the probabilities), we use the function `fit`, and then to make predictions for the emails of the test set, we use the function `predict`. However, both `fit` and `predict` take as input a data matrix. Hence, we first have to transform the list of emails to such a matrix. To do this, we will use the `CountVectorizer` object of `scikit-learn` (http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) which converts a collection of textual documents to a matrix of token counts. Therefore, at each iteration of the $k$-fold cross-validation, we first apply the `fit_transform` function of `CountVectorizer` to the emails of the training set. This transforms the text data (i.e. the emails of the training set) to the desired format. More specifically, the data is represented by the Email-Term matrix, where the rows correspond to the different emails of the training set and the columns to the features, which in our case are the different terms (i.e. words). Then, we use the `transform` function of `CountVectorizer` to the emails of the test set in order to transform them to an Email-Term matrix similarly as in the case of the training set. Then, we can train the Naive Bayes classifier and make predictions regarding the emails of the test set.

At each iteration, we also store the achieved accuracy of the classifier using the `accuracy_score` function of `scikit-learn` (http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html), and after the whole process has finished, we compute the average accuracy which demonstrates the overall performance of the classifier.

Your task is to write code that performs the $10$-fold cross-validation, and the learning and prediction tasks, and finally, to report on the accuracy of the Naive Bayes classifier. Next we provide the basic points and structure of the code for this task:
```python
# Import modules
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

nb = MultinomialNB() # Initialize Naive Bayes
skf = StratifiedKFold(n_splits=10) # Initialize k-fold
accuracy_scores = [] # List for storing accuracies

for train_indices, test_indices in skf.split(emails,y):
    
    # Body of for loop

print 'Total emails classified:', len(emails)
print 'Accuracy Score:', sum(accuracy_scores)/len(accuracy_scores)
```

In [4]:
#your code here


What is the accuracy achieved by the classifier? Did you expect that result?

---

Our next objective is to implement our own Naive Bayes classifier and use this classifier in the $k$-fold cross-validation process instead of the one provided by `scikit-learn`. Below you can find a function that takes as input a list of documents, and performs standard text processing tasks such as tokenization, stopword, punctuation and special character removal, and stemming. The function returns a list of processed documents.

In [4]:
import nltk
from nltk.stem.porter import *
import string

# Permorm data preprocessing
def text_preprocessing(docs):    
    # List of stopwords
    stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
                 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
                 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
                 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
                 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
                 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
                 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
                 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
                 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
                 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
                 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
                 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

    processed_docs = []
    
    for doc_id, text in enumerate(docs):
    
        # Remove punctuation and lowercase
        punct = re.compile(r'([^A-Za-z])')
        doc = punct.sub(" ", text)
        
        # Stopword removal
        doc = [w for w in doc.split() if w not in stopwords]  
        
        # Stemming
        stemmer = PorterStemmer()
        doc = [stemmer.stem(w) for w in doc] 
        
        # Covenrt list of words to one string
        doc = ' '.join(w for w in doc)
        processed_docs.append(doc)       
        
    return processed_docs

Use the function defined above to preprocess the emails of the dataset.

In [3]:
#your code here
processed_emails = text_preprocessing(emails)

Naive Bayes makes use of the Bayes rule to classify test examples. Recall the Bayes rule:

$$ P(y|x) = \frac{P(x|y)P(y)}{P(x)} $$

Let $C$ be the set of classes of the data. In our setting, $C=\{ spam, ham\}$. Naive Bayes computes the probability $P(y|x)$ for all $y \in C$ and classify $x$ into class $y' \in C$ such that:

$$ y' = argmax_{y \in C}P(y|x) $$

$P(x)$ is independent of $y$. It only scales $P(y|x)$. So $ P(y|x) \propto P(x|y)P(y) $ and therefore,

$$ y' = argmax_{y \in C}P(x|y)P(y) $$

which is all we need for $argmax_{y \in C}$

Our task is to classify emails as spam or ham. We have a lot of examples consisting of a list of words:

$$ \langle w_1,w_2,w_3,\ldots,w_n \rangle $$

Bayes theory tells us:	

$$ P(Spam|\langle w_1,\ldots,w_n \rangle) = \frac{P(\langle w_1,\ldots,w_n \rangle|Spam)P(Spam)}{P(\langle w_1,\ldots,w_n \rangle)}  $$

where $P(\langle w_1,\ldots,w_n \rangle) = P(\langle w_1,\ldots,w_n \rangle|Spam)P(Spam) + P(\langle w_1,\ldots,w_n \rangle|Ham)P(Ham)$

  
Computing $P(\langle w_1,\ldots,w_n \rangle|Spam)$ would require knowing (or estimating) the probability that every possible message is spam which is infeasible for real-world datasets. Naive Bayes solves this problem by making a very strong independence assumption:

$$ P(\langle w_1,\ldots,w_n \rangle|Spam) = \prod_{i=1}^n P(w_i|Spam) $$

Hence, to make our spam filter, we need only to estimate 
1. the priors $P(Spam)$ and $P(Ham)$
2. the likelihoods $P(w_i|Spam)$, $P(w_i|Ham)$ for all words $w_i$.

The priors $P(Spam)$, $P(Ham)$ are easy to compute:

$$ P(Spam) = \frac{\#(\text{spam messages})}{\#(\text{training documents})} $$
$$ P(Ham) = \frac{\#(\text{ham messages})}{\#(\text{training documents})} $$

The likelihoods $P(w_i|Spam)$, $P(w_i|Ham)$ for all words $w_i$ are also easy to estimate:

$$ P(w_i|Spam) = \frac{\#(\text{occurence of } w_i \text{ in spam messages})}{\#(\text{words in spam messages})} $$ $$P(w_i|Ham) = \frac{\#(\text{occurence of } w_i \text{ in ham messages})}{\#(\text{words in ham messages})} $$

However, the above equations are inappropriate for rare words. If term $w_i$ never appears in class $y$ in training data, then:

$$ P(w_i|y) = 0 $$

Therefore, documents containing $w_i$ have $0$ probability of being in class $y$ which causes overfitting to training data. To avoid this, we can apply Laplace smoothing:

$$ P(w_i|y) = \frac{\#(\text{occurence of } w_i \text{ in documents of class }y)+1}{\#(\text{words in documents of class }y)+|V|} $$

where $|V|$ is the number of terms in the vocabulary.

Your next task is to write a function that takes as input the preprocessed emails of the training set and their class labels ($1$ for ham and $-1$ for spam), and returns all the probabilities that are necessary to train a Naive Bayes classifier (i.e. (1) the priors $P(Spam)$ and $P(Ham)$, and (2) the likelihoods $P(w_i|Spam)$, $P(w_i|Ham)$ for all words $w_i$). The function should return the priors $P(Spam)$ (`pr_spam` variable) and $P(Ham)$ (`p_ham` variable) as well as a dictionary (`features`) containing the likelihoods $P(w_i|Spam)$, $P(w_i|Ham)$ for each word of the training emails. The basic structure of the code for this task is given below:
```python
def compute_probabilities(emails_train, y_train):
    
    # Body of the function
    
    return features, pr_ham, pr_spam
```

In [4]:
#your code here


Write a second function that takes as input a processed email, the priors $P(Spam)$ and $P(Ham)$, and the likelihoods $P(w_i|Spam)$, $P(w_i|Ham)$ for all words $w_i$ computed by the above function, and returns a prediction corresponding to the class of the input email ($1$ for ham and $-1$ for spam). The basic points and structure of the code for this function is shown below:
```python
# Import modules
from math import log

def predict(email, features, pr_spam, pr_ham):
    
    # Body of the function
    
    if ...:
        return 1
    else:
        return -1
```

For prediction, according to $ y' = argmax_{y \in C}P(x|y)P(y) $, you can use the following equations, as described above:

$$ P(\langle w_1,\ldots,w_n \rangle|Spam) = {\prod_{i=1}^n P(w_i|Spam)P(Spam)}  $$
$$ P(\langle w_1,\ldots,w_n \rangle|Ham) = {\prod_{i=1}^n P(w_i|Ham)P(Ham)} $$

In [5]:
#your code here

Run again a $k$-fold cross-validation process with $k$ = 10 to evaluate the performance of your own Naive Bayes classifier. Use the two functions defined above to compute the probabilities and to make predictions for the emails of the test set.

In [6]:
accuracy_scores = []

for train_indices, test_indices in skf.split(processed_emails,y):
    #your code here


print('Total emails classified:', len(emails))
print('Accuracy Score:', sum(accuracy_scores)/len(accuracy_scores))

Compare the accuracy achieved by your classifier with the one achieved by the classifier provided by `scikit-learn`.